# KoBERT-Pegasus 기반 한국어 요약 생성 노트북

이 노트북은 crawling_origin_with_summary.csv의 원본 텍스트를 KoBERT-Pegasus(pegasus-like) 모델로 요약하는 전체 파이프라인을 제공합니다.

In [ ]:
# 1. 환경 준비 및 필수 패키지 설치
import sys, subprocess

def install(package):
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', package])

for p in ['transformers', 'torch', 'pandas', 'tqdm', 'sentencepiece']:
    try:
        __import__(p.split('==')[0].split('>=')[0])
    except ImportError:
        install(p)

import pandas as pd, torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)

In [ ]:
# 2. 데이터 로드

def safe_read_csv(path):
    for enc in ['utf-8-sig', 'utf-8', 'cp949', 'euc-kr']:
        try: return pd.read_csv(path, encoding=enc)
        except: continue
    raise ValueError(f'CSV 파일 읽기 실패: {path}')

df = safe_read_csv('data/crawling_origin_with_summary.csv')
# 원본 텍스트 컬럼 자동 감지
orig_col = next((c for c in df.columns if '본문' in c or 'content' in c or '원본' in c), df.columns[0])
print('원본 텍스트 컬럼:', orig_col)
texts = df[orig_col].astype(str).fillna('').tolist()
print('총 샘플 수:', len(texts))

In [ ]:
# 3. KoBERT-Pegasus(pegasus-like) 모델 및 토크나이저 로드
# 대표적인 pegasus-like 한국어 모델: 'kakaobrain/kobart' 예시 사용
model_name = 'kakaobrain/kobart'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
print('모델 및 토크나이저 로드 완료:', model_name)

In [ ]:
# 4. 요약 생성 함수
def generate_summary(text, max_length=128):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512).to(device)
    with torch.no_grad():
        summary_ids = model.generate(**inputs, max_length=max_length, num_beams=3, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [ ]:
# 5. 전체 데이터 요약 생성 및 저장
summaries = []
for text in tqdm(texts, desc='요약 생성', mininterval=1.0):
    if not text or len(text.strip()) < 20:
        summaries.append('')
        continue
    try:
        summary = generate_summary(text)
    except Exception as e:
        summary = f'요약 실패: {str(e)[:30]}'
    summaries.append(summary)
df['pegasus_summary'] = summaries
df.to_csv('pegasus_summaries.csv', index=False, encoding='utf-8-sig')
print('요약 결과 저장 완료: pegasus_summaries.csv')

In [ ]:
# 6. 샘플 결과 확인
for i in range(3):
    print(f'원본: {texts[i][:100]}...')
    print(f'요약: {df["pegasus_summary"][i]}')
    print('-'*60)